In [1]:
from IPython.display import clear_output 

!pip install transformers==4.8.2
!pip install sentencepiece==0.1.96
!pip install tensorflow_addons

clear_output()

In [2]:
import sklearn
import tensorflow
import transformers
import tensorflow_addons

print(sklearn.__version__) # 1.0.2
print(tensorflow.__version__) # 2.8.0
print(transformers.__version__) # 4.8.2
print(tensorflow_addons.__version__) # 0.16.1

import sentencepiece

1.0.2
2.8.0
4.8.2
0.17.0


In [3]:
import pandas as pd
import numpy as np

import os
import re
import pickle 
import dill # for saving a function as a file(.pkl)
import logging # for changing the tf's logging level
import urllib.request
from tqdm import tqdm

from sklearn import model_selection
from sklearn.metrics import accuracy_score

import tensorflow as tf
import tensorflow_addons as tfa # for using Rectified-Adam optimizer (instead of Adam optimizer) 
from tensorflow.keras import layers, initializers, losses, optimizers, metrics, callbacks 

import transformers
from transformers import TFBertModel # BertTokenizer 제외

import sentencepiece as spm

In [ ]:
# Random seed 고정
tf.random.set_seed(1234)
np.random.seed(1234)

# Transformers logging level 변경 (WARNING -> ERROR) @ https://huggingface.co/transformers/main_classes/logging.html
transformers.logging.set_verbosity(transformers.logging.ERROR)

# Tensorflow logging level 변경 
tf.get_logger().setLevel(logging.ERROR)

# load data

In [ ]:
from google.colab import files
file_uploaded = files.upload()

Saving merged_dataset.csv to merged_dataset.csv


In [ ]:
df_1 = pd.read_csv('merged_dataset.csv', encoding='utf-8',index_col=0)
df_1.head()

,Sentence,label
0,배공빠르고 굿,2
1,택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고,1
2,아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다. 바느질이 조금 ...,2
3,선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다. 전...,1
4,민트색상 예뻐요. 옆 손잡이는 거는 용도로도 사용되네요 ㅎㅎ,2


In [ ]:
df_2 = pd.read_csv('text_data.csv', encoding='utf-8', index_col=0)
df_2.head()

,Sentence,Emotion
0,어깨라도넓엇으면 옷빨이라도 잘살텐데..,0
1,그저 보고싶어죽겟어요,0
2,아 진짜 저한테는 악운만 붙는거같네요ㅠㅠ........0,0
3,지금 계단에서울고있어요,0
4,저때문에 택배하나 배달하려다 변을 당하신건 아니겠죠0,0


In [ ]:
df_2.columns=['Sentence','label']

In [ ]:
dataset = pd.concat([df_1, df_2])
dataset.reset_index(inplace=True)

In [ ]:
dataset.head()

,index,Sentence,label
0,0,배공빠르고 굿,2
1,1,택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고,1
2,2,아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다. 바느질이 조금 ...,2
3,3,선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다. 전...,1
4,4,민트색상 예뻐요. 옆 손잡이는 거는 용도로도 사용되네요 ㅎㅎ,2


In [ ]:
del dataset['index']

In [ ]:
dataset.dropna(inplace=True)

In [ ]:
train_x, test_x, train_y, test_y = model_selection.train_test_split(dataset['Sentence'], dataset['label'], 
                                                                    test_size=0.3, 
                                                                    random_state=42) 

# preprocessing

In [ ]:
#KoBERT 불러오기
!git clone https://github.com/monologg/KoBERT-Transformers.git
!mv KoBERT-Transformers/kobert_transformers/tokenization_kobert.py /content
clear_output() # clear the output

#토크나이저 생성
from tokenization_kobert import KoBertTokenizer 

tokenizer = KoBertTokenizer.from_pretrained('monologg/kobert')

Downloading:   0%|          | 0.00/371k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/77.8k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

In [ ]:
dataset['Sentence'].str.len().sort_values(ascending=False)[:5]

567235    298
536769    260
539976    258
532135    253
535808    253
Name: Sentence, dtype: int64

In [ ]:
len(tokenizer.tokenize(dataset['Sentence'][567235]))

292

In [ ]:
SEQ_LEN = 300

token_ids = []
token_segments = []
token_masks = []

train_labels = []

for idx in tqdm(range(len(train_x))):

  # 'Sentence'칼럼에서 0번째 행~ 꺼내오기
  train_sentence = train_x.iloc[idx] 

  # 특수문자 제거
  cleaned_sentence = re.sub("[^\s0-9a-zA-Zㄱ-ㅎㅏ-ㅣ가-힣]", "", train_sentence)
  
  # 토큰화 + 토큰별 시퀀스번호 부여 + 패딩
  encoded_dict = tokenizer.encode_plus(text = cleaned_sentence,
                                       padding='max_length',
                                       truncation=True,
                                       max_length=SEQ_LEN
                                       )
  
  #'input_ids', 'token_type_ids', 'attention_mask'
  token_ids.append(encoded_dict['input_ids'])
  token_masks.append(encoded_dict['attention_mask'])
  token_segments.append(encoded_dict['token_type_ids'])

  train_labels.append(train_y.iloc[idx])


train_inputs = (np.array(token_ids), np.array(token_masks), np.array(token_segments))
train_labels = np.array(train_labels)

100%|██████████| 397694/397694 [01:59<00:00, 3330.15it/s]


# modeling

In [ ]:
def create_model(max_length=300):

    bert_base_model = TFBertModel.from_pretrained("monologg/kobert", from_pt=True) 

    input_token_ids   = layers.Input((max_length,), dtype=tf.int32, name='input_token_ids')   # tokens_tensor
    input_masks       = layers.Input((max_length,), dtype=tf.int32, name='input_masks')       # masks_tensor
    input_segments    = layers.Input((max_length,), dtype=tf.int32, name='input_segments')    # segments_tensor  

    bert_outputs = bert_base_model([input_token_ids, input_masks, input_segments]) 
    

    bert_outputs = bert_outputs[1] # ('pooler_output', <KerasTensor: shape=(None, 768) dtype=float32 (created by layer 'tf_bert_model')>)
    bert_outputs = layers.Dropout(0.2)(bert_outputs)
    final_output = layers.Dense(units=3, activation='softmax', kernel_initializer=initializers.TruncatedNormal(stddev=0.02), name="classifier")(bert_outputs)

    model = tf.keras.Model(inputs=[input_token_ids, input_masks, input_segments], 
                        outputs=final_output)

    model.compile(optimizer=tfa.optimizers.RectifiedAdam(learning_rate=1e-5, weight_decay=0.0025, warmup_proportion=0.05),
                  loss=losses.SparseCategoricalCrossentropy(), 
                  metrics=[metrics.SparseCategoricalAccuracy()])
    
    return model

In [ ]:
# TPU setting

resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
print("\nAll devices: ", tf.config.list_logical_devices('TPU'))

strategy = tf.distribute.TPUStrategy(resolver)

with strategy.scope(): 
    model = create_model(max_length=300)


All devices:  [LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:0', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:1', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:2', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:3', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:4', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:5', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:6', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:7', device_type='TPU')]


Downloading:   0%|          | 0.00/426 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/369M [00:00<?, ?B/s]

# Train

In [ ]:
# Checkpoint setting for saving the best model
from google.colab import drive
drive.mount('/gdrive')

checkpoint_path = '/gdrive/MyDrive/colab_data/temp_data/saved_models/'

if not os.path.exists(checkpoint_path):
    os.makedirs(checkpoint_path)

callback_checkpoint = callbacks.ModelCheckpoint(filepath=checkpoint_path + 'best_bert_weights.h5', 
                                                monitor='val_sparse_categorical_accuracy',
                                                save_best_only=True, 
                                                save_weights_only = True, 
                                                verbose=1) 
                                                
# Early-stopping for preventing the overfitting
callback_earlystop = callbacks.EarlyStopping(monitor='val_sparse_categorical_accuracy', 
                                             min_delta=0.0001, 
                                             patience=5) 

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [ ]:
history = model.fit(train_inputs, train_labels, validation_split=0.2,
                    epochs=10, batch_size=200,
                    verbose=1,
                    callbacks=[callback_checkpoint, callback_earlystop])

Epoch 1/10


INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'cond/Identity:0' shape=(None, 300) dtype=int64>, <tf.Tensor 'cond/Identity_8:0' shape=(None, 300) dtype=int64>, <tf.Tensor 'cond/Identity_16:0' shape=(None, 300) dtype=int64>, <tf.Tensor 'cond/Identity_24:0' shape=(None,) dtype=int64>]
INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'cond/Identity:0' shape=(None, 300) dtype=int64>, <tf.Tensor 'cond/Identity_8:0' shape=(None, 300) dtype=int64>, <tf.Tensor 'cond/Identity_16:0' shape=(None, 300) dtype=int64>, <tf.Tensor 'cond/Identity_24:0' shape=(None,) dtype=int64>]


3182/3182 [==============================] - ETA: 0s - loss: 0.4862 - sparse_categorical_accuracy: 0.7952

INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'cond/Identity:0' shape=(None, 300) dtype=int64>, <tf.Tensor 'cond/Identity_8:0' shape=(None, 300) dtype=int64>, <tf.Tensor 'cond/Identity_16:0' shape=(None, 300) dtype=int64>, <tf.Tensor 'cond/Identity_24:0' shape=(None,) dtype=int64>]



Epoch 1: val_sparse_categorical_accuracy improved from -inf to 0.85807, saving model to /gdrive/MyDrive/colab_data/temp_data/saved_models/best_bert_weights.h5
3182/3182 [==============================] - 1168s 329ms/step - loss: 0.4862 - sparse_categorical_accuracy: 0.7952 - val_loss: 0.3572 - val_sparse_categorical_accuracy: 0.8581
Epoch 2/10
3182/3182 [==============================] - ETA: 0s - loss: 0.3404 - sparse_categorical_accuracy: 0.8651
Epoch 2: val_sparse_categorical_accuracy improved from 0.85807 to 0.86708, saving model to /gdrive/MyDrive/colab_data/temp_data/saved_models/best_bert_weights.h5
3182/3182 [==============================] - 970s 305ms/step - loss: 0.3404 - sparse_categorical_accuracy: 0.8651 - val_loss: 0.3366 - val_sparse_categorical_accuracy: 0.8671
Epoch 3/10
3182/3182 [==============================] - ETA: 0s - loss: 0.2936 - sparse_categorical_accuracy: 0.8851
Epoch 3: val_sparse_categorical_accuracy improved from 0.86708 to 0.87495, saving model to /g

# Evaluate

In [ ]:
SEQ_LEN = 300 # 최대 token 개수 이상의 값으로 임의로 설정

token_ids =[]
token_masks =[]
token_segments =[]

test_labels = []


for idx in tqdm(range(len(test_x))):

    test_sentence = test_x.iloc[idx]
    
    # 특수문자 제거
    cleaned_sentence = re.sub("[^\s0-9a-zA-Zㄱ-ㅎㅏ-ㅣ가-힣]", "", test_sentence) 
    
    # Tokenizing / Tokens to sequence numbers / Padding
    encoded_dict = tokenizer.encode_plus(text=cleaned_sentence,
                                         padding='max_length', 
                                         truncation=True,
                                         max_length=SEQ_LEN)
    
    token_ids.append(encoded_dict['input_ids']) # tokens_tensor
    token_masks.append(encoded_dict['attention_mask']) # masks_tensor
    token_segments.append(encoded_dict['token_type_ids']) # segments_tensor

    test_labels.append(test_y.iloc[idx])


test_inputs = (np.array(token_ids), np.array(token_masks), np.array(token_segments))
test_labels = np.array(test_labels)

100%|██████████| 170441/170441 [01:03<00:00, 2690.95it/s]


In [ ]:
model.load_weights(filepath=checkpoint_path + 'best_bert_weights.h5') # Load the best model's weights from checkpoint file

preds = model.predict(test_inputs)
preds = tf.argmax(preds, axis=1)

print("Accuracy: ",accuracy_score(preds, test_labels))

INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'cond/Identity:0' shape=(None, 300) dtype=int64>, <tf.Tensor 'cond/Identity_8:0' shape=(None, 300) dtype=int64>, <tf.Tensor 'cond/Identity_16:0' shape=(None, 300) dtype=int64>]


Accuracy:  0.8774355935485006


In [ ]:
from sklearn.metrics import precision_score , recall_score , confusion_matrix, classification_report

target_names = ['Neutral','Positive','Negative']

precision = precision_score(test_labels, preds,average= "macro")
recall = recall_score(test_labels, preds,average= "macro")

print("< Confusion Matrix >\n\n",confusion_matrix(test_labels, preds))
print("\n")
print("< Classification Report >\n\n",classification_report(test_labels, preds, target_names=target_names))

< Confusion Matrix >

 [[13674   922  1017]
 [ 1331 66924  8931]
 [ 1290  7399 68953]]


< Classification Report >

               precision    recall  f1-score   support

     Neutral       0.84      0.88      0.86     15613
    Positive       0.89      0.87      0.88     77186
    Negative       0.87      0.89      0.88     77642

    accuracy                           0.88    170441
   macro avg       0.87      0.88      0.87    170441
weighted avg       0.88      0.88      0.88    170441



# Save Model

In [ ]:
from google.colab import drive
drive.mount('/gdrive')


data_path = '/gdrive/MyDrive/colab_data/temp_data/' 

if not os.path.exists(data_path): 
    os.makedirs(data_path)


with open(data_path + 'model_BERTfunction_v1.pkl', 'wb') as f:
    pickle.dump(dill.dumps(create_model), f) # use dill to pickle a function (https://j.mp/3CeSIzP & https://j.mp/3AaXxYW)

with open(data_path + 'tokenizer-bert.pkl', 'wb') as f:
    pickle.dump(tokenizer, f)     

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


Load

In [4]:
!pip install transformers==4.8.2
!pip install tensorflow_addons

import pandas as pd
import numpy as np

import re
import os
import pickle 
import dill # for saving a function as a file
import logging # for changing the tf's logging level
from IPython.display import clear_output 

import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow.keras import layers, initializers, losses, optimizers, metrics 

import transformers
from transformers import TFBertModel 

# 이번 실습에서 추가되었습니다
!pip install sentencepiece==0.1.96
import sentencepiece as spm 
!git clone https://github.com/monologg/KoBERT-Transformers.git 
!mv KoBERT-Transformers/kobert_transformers/tokenization_kobert.py /content 
from tokenization_kobert import KoBertTokenizer 

# Tensorflow logging level 변경 
tf.get_logger().setLevel(logging.ERROR)

# clear the output after the installation
clear_output() 

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# 1) Load the Model-builder function
with open("/content/drive/MyDrive/final/colab_data/temp_data/model_BERTfunction_v1.pkl", 'rb') as f: # 이번 실습에서 변경되었습니다
    create_model = dill.loads(pickle.load(f)) # use dill to pickle a python function

In [10]:
# 2) Load the Bert-tokenizer 
with open("/content/drive/MyDrive/final/colab_data/temp_data/tokenizer-bert.pkl", 'rb') as f:
    tokenizer = pickle.load(f) 

In [11]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.TPUStrategy(resolver) # Obsolete : tf.distribute.experimental.TPUStrategy()

with strategy.scope(): 
    model = create_model(max_length=300) 

All PyTorch model weights were used when initializing TFBertModel.

All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [12]:
checkpoint_path = '/content/drive/MyDrive/final/colab_data/temp_data/saved_models/'
model.load_weights(filepath=checkpoint_path + 'best_bert_weights.h5')

In [13]:
def predict_sentiment(sentence, tokenizer, model):
    
    SEQ_LEN = 300 

    # Tokenizing / Tokens to sequence numbers / Padding
    encoded_dict = tokenizer.encode_plus(text=re.sub("[^\s0-9a-zA-Zㄱ-ㅎㅏ-ㅣ가-힣]", "", sentence),
                                         padding='max_length', 
                                         truncation = True,
                                         max_length=SEQ_LEN) 
    
    token_ids = np.array(encoded_dict['input_ids']).reshape(1, -1) 
    token_masks = np.array(encoded_dict['attention_mask']).reshape(1, -1)
    token_segments = np.array(encoded_dict['token_type_ids']).reshape(1, -1)
    
    new_inputs = (token_ids, token_masks, token_segments)

    # Prediction
    prediction = model.predict(new_inputs)
    predicted_probability = np.round(np.max(prediction) * 100, 2) 
    predicted_class = ['중립','부정', '긍정'][np.argmax(prediction, axis=1)[0]] 
    
    print("{}% 확률로 {} 리뷰입니다.".format(predicted_probability, predicted_class))

In [14]:
song="이윽고 내가 한눈에너를 알아봤을 때모든 건 분명 달라지고 있었어내 세상은 널 알기 전과 후로 나뉘어니가 숨 쉬면 따스한 바람이 불어와니가 웃으면 눈부신 햇살이 비춰거기 있어줘서 그게 너라서가끔 내 어깨에 가만히 기대주어서나는 있잖아 정말 빈틈없이 행복해너를 따라서 시간은 흐르고 멈춰물끄러미 너를 들여다 보곤 해그것 말고는 아무것도 할 수 없어서너의 모든 순간 그게나였으면 좋겠다생각만 해도 가슴이 차올라나는 온통 너로보고 있으면 왠지 꿈처럼 아득한 것몇 광년 동안 날 향해 날아온 별빛 또 지금의 너거기 있어줘서 그게 너라서가끔 나에게 조용하게 안겨주어서나는 있잖아 정말 남김없이 고마워너를 따라서 시간은 흐르고 멈춰물끄러미 너를 들여다보곤 해너를 보는 게 나에게는 사랑이니까너의 모든 순간 그게나였으면 좋겠다생각만 해도 가슴이 차올라나는 온통 너로니 모든 순간 나였으면"

In [16]:
song1="네가 없이 웃을 수 있을까생각만 해도 눈물이나힘든 시간 날 지켜준 사람이제는 내가 그댈 지킬 테니너의 품은 항상 따뜻했어고단했던 나의 하루에유일한 휴식처나는 너 하나로 충분해긴 말 안 해도 눈빛으로 다 아니깐한 송이의 꽃이 피고 지는모든 날 모든 순간 함께해햇살처럼 빛나고 있었지나를 보는 네 눈빛은꿈이라고 해도 좋을 만큼그 모든 순간은 눈부셨다불안했던 나의 고된 삶에한줄기 빛처럼 다가와날 웃게 해준 너나는 너 하나로 충분해긴 말 안 해도 눈빛으로 다 아니깐한 송이의 꽃이 피고 지는모든 날 모든 순간 함께해알 수 없는 미래지만네 품속에 있는 지금 순간 순간이영원 했으면 해갈게 바람이 좋은 날에햇살 눈부신 어떤 날에 너에게로처음 내게 왔던 그날처럼모든 날 모든 순간 함께해"

In [15]:
predict_sentiment(song, tokenizer, model)

INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'cond/Identity:0' shape=(None, 300) dtype=int64>, <tf.Tensor 'cond/Identity_8:0' shape=(None, 300) dtype=int64>, <tf.Tensor 'cond/Identity_16:0' shape=(None, 300) dtype=int64>]


94.69% 확률로 긍정 리뷰입니다.


In [17]:
predict_sentiment(song1, tokenizer, model)

96.3% 확률로 긍정 리뷰입니다.


In [18]:
song2="일부러 몇 발자국 물러나내가 없이 혼자 걷는 널 바라본다옆자리 허전한 너의 풍경흑백 거리 가운데 넌 뒤돌아본다그때 알게 되었어난 널 떠날 수 없단 걸우리 사이에 그 어떤 힘든 일도이별보단 버틸 수 있는 것들이었죠어떻게 이별까지 사랑"

In [19]:
predict_sentiment(song2, tokenizer, model)

97.33% 확률로 긍정 리뷰입니다.


In [20]:
song3="너는 참 못됐다나 없이도 잘 지내더라언제쯤 나도 널 잊을 수 있을까눈을 감아도 눈물이 흐르고어딜 가도 너만 보여어느새 니가 없는긴 하루가 지나간다다른 사람을 또 만나봐도니가 아님 안 되겠어울다가 또 웃다가 미친 것처럼또 보고 싶다상처 같은 널 잊어보려고아무리 애를 써도너만 보고 싶다니가 없인 아무것도 못 해니가 없는 나는 아무것도 아냐나도 참 못됐다자꾸만 또 힘이 들더라예전처럼 돌아갈 수가 있을까눈을 감아도 니가 떠오르고어딜 가도 너만 보여자주 걷던 그 길 끝 저 너머그댄 보일까요다른 사람을 또 만나봐도니가 아님 안 되겠어울다가 또 웃다가 미친 것처럼또 보고 싶다상처 같은 널 잊어보려고아무리 애를 써도너만 보고 싶다니가"

In [21]:
predict_sentiment(song3, tokenizer, model)

94.23% 확률로 긍정 리뷰입니다.
